# Решение #1 - Фильтрация

### Импортирование библиотек и загрузка датасета

In [1]:
from src import *
import pandas as pd
from tqdm.auto import tqdm
from ipywidgets import FloatProgress

df = pd.read_json('./../data/sample.json')

### Структура датасета

In [2]:
df.head()

,id,text
0,1,Ты нашёл их или нет?
1,2,Почему она так со мной поступает?
2,3,Никто туда больше не ходит.
3,4,У него с собой не было тогда денег.
4,5,Почему они с нами так поступают?


### Предобработка строк

In [3]:
df['text_preprocess'] = df['text'].apply(load_dataset.preprocess_text)

In [4]:
df.head()

,id,text,text_preprocess
0,1,Ты нашёл их или нет?,ты нашел их или нет
1,2,Почему она так со мной поступает?,почему она так со мной поступает
2,3,Никто туда больше не ходит.,никто туда больше не ходит
3,4,У него с собой не было тогда денег.,у него с собой не было тогда денег
4,5,Почему они с нами так поступают?,почему они с нами так поступают


### Решение основано на поэтапном отборе групп
Применяются следующие алгоритмы:
* Растояние Карловского
* Сравнение множеств слов
* Расстояние Хэмминга
* Косинусное сходство с векторизацией TF-IDF
* Сходство Джаро-Винклера
* Сравнение наборов местоимений


In [5]:
pd.options.mode.chained_assignment = None # Убираем лишние предупреждения
df['used'] = [False] * len(df)

# Функция фильтрации на основе алгоритма верификации рерайтов (function) и заданного ограничения (threshold) 
# c возможностью проверки наборов местоимений (check_pronoun) 
def filter(groups,function,threshold,check_pronoun=True):
    for i in tqdm(range(len(df))):
        if df['used'][i]:  # Проверка метки использования строки
            continue
        groups.append([])
        groups[-1].append(df['text'][i])
        df['used'][i] = True
        
        for j in range(i+1,len(df)):
            if df['used'][j]: # Проверка метки использования строки
                continue
            if check_pronoun:
                if(function(df['text_preprocess'][i],df['text_preprocess'][j],threshold) and pronouns.check_pronoun_correspondence(df['text_preprocess'][i],df['text_preprocess'][j])):
                    groups[-1].append(df['text'][j])
                    df['used'][j] = True
            else:
                if(function(df['text_preprocess'][i],df['text_preprocess'][j],threshold)):
                    groups[-1].append(df['text'][j])
                    df['used'][j] = True
        if len(groups[-1]) == 1: # Если строка не имеет рерайта, то группа не формируется
            df['used'][i] = False 
            groups = groups[:-1]
    return groups

groups = []
# Фильтрация на основе алгоритмов верификации рерайтов
groups = filter(groups,karlovskiy_distance.is_rewrite_karlovskiy_distance,0.89,check_pronoun=True)
groups = filter(groups,word_set.is_rewrite_word_set,threshold=0.99,check_pronoun=False)
groups = filter(groups,hamming_distance_custom.is_rewrite_hamming_distance_custom,threshold=0.9,check_pronoun=False)
groups = filter(groups,cosine_tfidf.is_rewrite_cosine_tfidf,0.7,check_pronoun=True)
groups = filter(groups,jaro_winkler.is_rewrite_jaro_winkler,0.9,check_pronoun=True)

# Добавляем уникальные строки (не имеющие рерайт)
groups = groups + [[str] for str in list(df[df['used']==False]['text'])]

df = df.drop('used',axis=1)

  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/412 [00:00<?, ?it/s]

  0%|          | 0/412 [00:00<?, ?it/s]

### Количество найденных групп

In [6]:
len(groups)

277

### Все найденные группы

In [7]:
groups

[['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влера письмо.', 'Я написал вчера письмо.'],
 ['Мы не были готовы.', 'Мы были не готовы.'],
 ['Они их только что нашли.', 'Они только что их нашли.'],
 ['

### Строки, которые имеют более 1 рерайта

In [8]:
groups3 = [group for group in groups if len(group) > 2]

In [9]:
len(groups3)

1

In [10]:
groups3

[['Том не хочет никого видеть.',
  'Том не хочет сегодня никого видеть.',
  'Том сегодня не хочет никого видеть.']]

### Строки, которые имеют ровно 1 рерайт

In [11]:
groups2 = [group for group in groups if len(group) == 2]

In [12]:
len(groups2)

133

In [13]:
groups2

[['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влера письмо.', 'Я написал вчера письмо.'],
 ['Мы не были готовы.', 'Мы были не готовы.'],
 ['Они их только что нашли.', 'Они только что их нашли.'],
 ['

### Строки, которые не имеют рерайта

In [14]:
groups1 = [group for group in groups if len(group) == 1]

In [15]:
len(groups1)

143

In [16]:
groups1

[['Ты нашёл их или нет?'],
 ['Почему она так со мной поступает?'],
 ['Почему они с нами так поступают?'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Почему она так с ней поступает?'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Том этим сейчас занимается.'],
 ['Том был просто не готов.'],
 ['Тому это тоже не нравится.'],
 ['Сколько сейчас времени в Париже?'],
 ['Тому было больше не к кому обратиться.'],
 ['Он сказал, что вчера был дома.'],
 ['Они никогда меня не слушают.'],
 ['Я поймал сегодня три рыбы.'],
 ['Я хочу что-нибудь сделать для Тома.'],
 ['Я сделал всё правильно?'],
 ['Я тоже завтра пойду в университет.'],
 ['Мне никогда не нужно было столько денег.'],
 ['Том не может жить без Мэри.'],
 ['Почему они так с ним поступают?'],
 ['Что могло с ними случиться?'],
 ['Мы будем делать всё по-своему.'],
 ['Где был Том весь день?'],
 ['Я здесь часто ем.'],
 ['Я всё ей рассказываю.'],
 [

### Сохранение результата

In [17]:
save_groups.save_groups(groups,"../output/solution1-result.json")